In [1]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import requests
import pandas as pd

C:\Users\44771\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\airflow\__init__.py:36: RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.
  warnings.warn(
OSError while attempting to symlink the latest log directory


C:\Users\44771\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\airflow\plugins_manager.py:30 DeprecationWarning: 'cgitb' is deprecated and slated for removal in Python 3.13

C:\Users\44771\AppData\Local\Temp\ipykernel_30320\134980821.py:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

In [2]:
def fetch_weather_data():
    api_key = 'ab2ed3d3edb04581b6a151349242008'
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q=London"

    # Parameters to be sent with the GET request
    params = {
        'key': api_key,         # Your API key
        'q': 'London'       # The location for which you want the weather data
    }

    # Making the GET request
    response = requests.get(url, params=params)

    # Handling the response
    if response.status_code == 200:
        # If the request was successful, print the JSON data
        data = response.json()
        data
    else:
        # If the request failed, print the status code
        print(f"Request failed with status code: {response.status_code}")

    all_data = data['current']

    condition = all_data['condition']
    condition.pop('icon')
    condition.pop('code')

    all_data.pop('condition')

    weather_df = pd.DataFrame([all_data])
    condition_df = pd.DataFrame([condition])

    weather_df = pd.concat([weather_df,condition_df],axis=1)
    weather_df.rename(columns = {'text':'description'},inplace=True)

    weather_df.to_csv('weather_data.csv', mode='a', header=False, index=False)

# Define the default arguments for the DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Define the DAG
dag = DAG(
    'weather_data_dag',
    default_args=default_args,
    description='Fetch weather data every few minutes',
    schedule_interval=timedelta(minutes=15),  # Run every 15 minutes
    start_date=datetime(2024, 1, 1),
    catchup=False,
)

# Define the task
fetch_weather_task = PythonOperator(
    task_id='fetch_weather_data',
    python_callable=fetch_weather_data,
    dag=dag,
)

fetch_weather_task

C:\Users\44771\AppData\Local\Temp\ipykernel_30320\242979740.py:50 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<Task(PythonOperator): fetch_weather_data>